In [31]:
import ARXT
import time
import bayesian_changepoint_detection.bayesian_models as bayes_models
from bayesian_changepoint_detection.hazard_functions import constant_hazard
import bayesian_changepoint_detection.online_likelihoods as online_ll
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from math import sqrt
from ARXT import hit_rate



In [102]:
def get_data(differencing = False):

    # tickers = ["^GSPC", "^IXIC", "^DJI","JPYUSD=X", "^VIX", "GBPUSD=X", "EURUSD=X] # , "GBPUSD=X", "EURUSD=X",
    # data = Data_gen.collect_data(tickers)
    data = pd.read_csv("../Data/fin_data.csv")
    # data.to_csv("Data/fin_data.csv")
    data.index = data["Date"]
    data = data.drop(["Date"], axis=1)
    data.columns = range(data.shape[1])
    print(differencing)
    if differencing:
        data = data.pct_change()
        data = data.iloc[1:]
    return(data)


In [36]:
DATA = get_data(differencing=True)
p = 5
train = True

start_time = time.time()
train_len = 1000
AR = ARXT.AR_p(DATA, p)    

AR.AR_p_model(0, train_len)
c_det = bayes_models.OnlineChagepoint(np.array(DATA[0]), constant_hazard, 200)
log_likelihood_class = c_det.warm_run(llc = online_ll.StudentT(alpha=0.1, beta=.01, kappa=1, mu=0),t = train_len)
Nw = 200
forecasts = []
retraining_points = []
for i in range(train_len, len(DATA[0])):
    if train:
        if c_det.iteration(i, log_likelihood_class, Nw):
            print("retraining at ", DATA.index[i])
            retraining_points.append(DATA.index[i])
            AR.AR_p_model(max(0,i-500), i)

    forecasts.append(AR.predict(DATA[i-p:i]))

# plt.plot(DATA.iloc[train_len:,0], label="truth")
# plt.plot(forecasts, label="forecasts")
# plt.legend()
# plt.show()
retrain = ""
if train: retrain = "retune"
end_time = time.time()
duration = end_time-start_time
print("Time taken for AR(p) {}: {}".format(retrain, duration))
pd.DataFrame(forecasts).to_csv("forecasts_AR_{}.csv".format(retrain))

retraining at  2007-08-09
retraining at  2021-01-12
Time taken for AR(p) retune: 9.786260843276978


In [104]:
DATA = get_data(differencing=True)
DATA

True


,0,1,2,3,4,5,6,7,8
Date,,,,,,,,,
2003-01-02,0.033200,-0.012478,0.031875,0.036945,-0.112858,0.028072,0.023364,-0.005716,0.000000
2003-01-03,-0.000484,0.004512,-0.000677,0.001610,-0.027964,0.003065,-0.043233,-0.005749,-0.012195
2003-01-06,0.022474,0.002597,0.019982,0.024685,0.009319,0.085370,0.031986,-0.016625,0.006173
2003-01-07,-0.006545,-0.007154,-0.003759,0.007212,0.008832,-0.013479,0.022877,-0.005513,-0.006135
2003-01-08,-0.014086,0.009319,-0.016621,-0.021305,0.015917,-0.024559,-0.031323,-0.005543,0.055556
...,...,...,...,...,...,...,...,...,...
2022-12-26,0.000000,-0.003261,0.000000,0.000000,0.000000,-0.002700,0.002481,-0.008725,0.000000
2022-12-27,-0.004050,0.000196,0.001133,-0.013777,0.037374,-0.000268,-0.004697,-0.004739,0.027273
2022-12-28,-0.012021,-0.005409,-0.011006,-0.013517,0.022633,-0.000714,0.000295,-0.004762,0.008850


In [107]:
df = get_data(differencing=True)
for i in df.columns[1:]:
    for lag in range(1,p):

        df[f'{i}_{lag}'] = df[i].shift(lag)

# df = df.dropna(inplace=True, axis=1)

df = df.iloc[p:,:]
df

True


,0,1,2,3,4,5,6,7,8,1_1,...,6_3,6_4,7_1,7_2,7_3,7_4,8_1,8_2,8_3,8_4
Date,,,,,,,,,,,,,,,,,,,,,
2003-01-09,0.019386,-0.002262,0.021043,0.026687,-0.050137,-0.028723,-0.025819,-0.005574,0.040936,0.009319,...,0.031986,-0.043233,-0.005543,-0.005513,-0.016625,-0.005749,0.055556,-0.006135,0.006173,-0.012195
2003-01-10,0.000000,0.000671,0.000992,0.006437,0.002887,0.068821,0.062054,-0.005605,-0.016854,-0.002262,...,0.022877,0.031986,-0.005574,-0.005543,-0.005513,-0.016625,0.040936,0.055556,-0.006135,0.006173
2003-01-13,-0.001412,0.000503,0.000124,-0.001160,0.023849,-0.100171,-0.040372,0.012401,0.011429,0.000671,...,-0.031323,0.022877,-0.005605,-0.005574,-0.005543,-0.005513,-0.016854,0.040936,0.055556,-0.006135
2003-01-14,0.005830,0.010508,0.006447,0.010339,-0.013253,0.002657,0.002876,0.008909,-0.005650,0.000503,...,-0.025819,-0.031323,0.012401,-0.005605,-0.005574,-0.005543,0.011429,-0.016854,0.040936,0.055556
2003-01-15,-0.014426,-0.000931,-0.013507,-0.015188,0.038258,-0.001325,0.000000,0.008830,0.022727,0.010508,...,0.062054,-0.025819,0.008909,0.012401,-0.005605,-0.005574,-0.005650,0.011429,-0.016854,0.040936
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-26,0.000000,-0.003261,0.000000,0.000000,0.000000,-0.002700,0.002481,-0.008725,0.000000,0.000098,...,-0.009238,0.002991,0.000672,0.000672,0.000672,0.000673,0.004566,-0.035242,0.013393,0.041860
2022-12-27,-0.004050,0.000196,0.001133,-0.013777,0.037374,-0.000268,-0.004697,-0.004739,0.027273,-0.003261,...,-0.003014,-0.009238,-0.008725,0.000672,0.000672,0.000672,0.000000,0.004566,-0.035242,0.013393
2022-12-28,-0.012021,-0.005409,-0.011006,-0.013517,0.022633,-0.000714,0.000295,-0.004762,0.008850,0.000196,...,0.001080,-0.003014,-0.004739,-0.008725,0.000672,0.000672,0.027273,0.000000,0.004566,-0.035242


In [118]:
from statsmodels.tsa.arima.model import ARIMA


model = ARIMA(endog=df.iloc[:1000,0], exog=df.iloc[:1000,1:], order=(p,1,0))
model_fit = model.fit()


In [114]:
prediction_temp = []
for i in range(1000):
    prediction_temp.append(model_fit.forecast(exog=df.iloc[i-1,1:])[0])

In [115]:
print(hit_rate(df.iloc[1000:2000,0], prediction_temp))
print(hit_rate(df.iloc[1000:2000,0], forecasts[:1000]))

0.48348348348348347
0.5015015015015015


In [116]:
print(sqrt(mean_squared_error(df.iloc[1000:2000,0], prediction_temp)))
print(sqrt(mean_squared_error(df.iloc[1000:2000,0], forecasts[:1000])))

0.019109453048249254
0.02346021434730719


In [119]:
model_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                 1000
Model:                 ARIMA(5, 1, 0)   Log Likelihood                5124.972
Date:                Fri, 01 Dec 2023   AIC                         -10157.944
Time:                        09:40:21   BIC                          -9932.234
Sample:                    01-09-2003   HQIC                        -10072.154
                         - 11-08-2006                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
1              0.0087      0.008      1.091      0.275      -0.007       0.024
2              0.6889      0.011     62.494      0.000       0.667       0.711
3              0.2319      0.008     28.267      0.000       0.216       0.248
4             -0.0051      0.001     -3.872      0.000      -0.008      -0.003
5             -0.0038      0.004     -1.047      0.295      -0.011       0.003
6              0.0073      0.004      1.810      0.070      -0.001       0.015
7              0.0017      0.008      0.211      0.833      -0.014       0.018
8              0.0034      0.004      0.878      0.380      -0.004       0.011
1_1            0.0076      0.008      0.910      0.363      -0.009       0.024
1_2           -0.0343      0.009     -3.876      0.000      -0.052      -0.017
1_3            0.0102      0.009      1.139      0.255      -0.007       0.028
1_4            0.0045      0.009      0.519      0.604      -0.013       0.022
2_1           -0.0436      0.012     -3.572      0.000      -0.067      -0.020
2_2           -0.0011      0.011     -0.093      0.926      -0.024       0.021
2_3           -0.0070      0.012     -0.592      0.554      -0.030       0.016
2_4           -0.0118      0.011     -1.036      0.300      -0.034       0.011
3_1            0.0060      0.008      0.741      0.459      -0.010       0.022
3_2            0.0016      0.008      0.200      0.841      -0.014       0.018
3_3           -0.0057      0.008     -0.684      0.494      -0.022       0.011
3_4            0.0038      0.008      0.481      0.631      -0.012       0.020
4_1           -0.0026      0.001     -2.237      0.025      -0.005      -0.000
4_2            0.0005      0.001      0.423      0.672      -0.002       0.003
4_3           -0.0027      0.001     -2.236      0.025      -0.005      -0.000
4_4           -0.0023      0.001     -1.661      0.097      -0.005       0.000
5_1            0.0002      0.004      0.060      0.953      -0.007       0.007
5_2            0.0004      0.003      0.123      0.902      -0.006       0.007
5_3           -0.0055      0.004     -1.555      0.120      -0.012       0.001
5_4            0.0010      0.003      0.291      0.771      -0.006       0.008
6_1           -0.0014      0.004     -0.342      0.732      -0.009       0.007
6_2           -0.0007      0.004     -0.172      0.864      -0.008       0.007
6_3            0.0065      0.004      1.556      0.120      -0.002       0.015
6_4           -0.0040      0.004     -1.004      0.315      -0.012       0.004
7_1            0.0047      0.008      0.574      0.566      -0.011       0.021
7_2           -0.0054      0.009     -0.611      0.541      -0.023       0.012
7_3            0.0038      0.008      0.486      0.627      -0.012       0.019
7_4           -0.0013      0.007     -0.176      0.860      -0.015       0.013
8_1           -0.0065      0.003     -1.865      0.062      -0.013       0.000
8_2           -0.0046      0.004     -1.102      0.270      -0.013       0.004
8_

In [28]:
DATA.iloc[1001,0]

-0.0022232342125417803

In [33]:
DATA.iloc[:1000,1:]

,1,2,3,4,5,6,7,8
Date,,,,,,,,
2003-01-02,-0.012478,0.031875,0.036945,-0.112858,0.028072,0.023364,-0.005716,0.000000
2003-01-03,0.004512,-0.000677,0.001610,-0.027964,0.003065,-0.043233,-0.005749,-0.012195
2003-01-06,0.002597,0.019982,0.024685,0.009319,0.085370,0.031986,-0.016625,0.006173
2003-01-07,-0.007154,-0.003759,0.007212,0.008832,-0.013479,0.022877,-0.005513,-0.006135
2003-01-08,0.009319,-0.016621,-0.021305,0.015917,-0.024559,-0.031323,-0.005543,0.055556
...,...,...,...,...,...,...,...,...
2006-10-26,0.004731,0.002388,0.009552,-0.009381,0.006105,0.006122,0.006664,-0.008547
2006-10-27,0.006804,-0.006034,-0.011971,0.022727,0.003861,0.004127,0.006620,-0.004310
2006-10-30,-0.000255,-0.000311,0.005594,0.037037,-0.001884,0.000790,-0.001315,-0.008658


In [35]:
DATA.iloc[1000:2000,1:]

,1,2,3,4,5,6,7,8
Date,,,,,,,,
2006-11-02,0.001025,-0.001037,-0.000141,-0.007819,-0.018374,-0.012513,-0.003982,0.000000
2006-11-03,-0.007459,-0.002704,-0.001384,-0.022767,-0.002686,-0.010378,-0.003998,0.017544
2006-11-06,-0.001608,0.009971,0.015085,0.000000,0.061606,0.065531,-0.011151,0.012931
2006-11-07,0.004932,0.004231,0.004197,-0.006272,0.011971,0.003045,-0.003608,-0.004255
2006-11-08,-0.002037,0.001626,0.003813,-0.030658,-0.004074,0.024612,-0.003622,0.004274
...,...,...,...,...,...,...,...,...
2010-08-26,-0.005975,-0.007381,-0.010670,0.025094,0.003301,0.003038,0.000920,0.013072
2010-08-27,0.004007,0.016508,0.016491,-0.106686,0.002046,-0.000322,0.000919,0.038710
2010-08-30,-0.013509,-0.013883,-0.015629,0.112883,-0.005498,-0.003095,-0.008267,-0.037267
